# Control Example "Add Loads"

Manipulate a grid in PowerFactory by adding new loads to selected terminals. Then run the automatic layout command to draw the related grid graphics.

Instructions:
- Import the provided PowerFactory example (examples/grids/*.pfd) in PowerFactory (if not already done)
- Close the PowerFactory Application
- Run this example
    - Initiate the compatible Interface
    - Execute control action using a controller instance

General remark for debugging with the PowerFactory GUI:
- $ pfi.app.Show() $ opens the PowerFactory GUI in non-interavtive mode
- $ pfi.app.Hide() $ closes PowerFactory GUI (do not close the window by your own by clicking on the red cross)

### Select an compatible PowerFactory Interface
- Import the PowerFactoryInterface (in this example: that is compatible with PowerFactory in version 2024)
- Specify additional PowerFactory configuration
- Name the PowerFactory project

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import math
import pathlib
import time
import typing as t
from collections.abc import Sequence

from powerfactory_tools.versions.pf2024 import PowerFactoryInterface
from powerfactory_tools.versions.pf2024.interface import ValidPFValue
from powerfactory_tools.versions.pf2024.interface import ValidPythonVersion
from powerfactory_tools.versions.pf2024.types import CalculationCommand
from powerfactory_tools.versions.pf2024.types import PFClassId
from powerfactory_tools.versions.pf2024.types import PowerFactoryTypes as PFTypes

# PowerFactory configuration
PF_SERVICE_PACK = 2  # mandatory
PF_USER_PROFILE = ""  # specification may be necessary
PF_INI_NAME = ""  # optional specification of ini file name to switch to full version (e.g. PowerFactoryFull for file PowerFactoryFull.ini)
PF_PYTHON_VERSION = ValidPythonVersion.VERSION_3_12  # python version of local code environment must match the python version of PowerFactory API


# Consider to use raw strings to avoid misinterpretation of special characters, e.g. r"dir\New Project" or r"dir\1-HV-grid".
PROJECT_NAME = "PowerFactory-Tools"  # may be also full path "dir_name\\project_name"
EXPORT_PATH = pathlib.Path("control_action_results")

### Define control routine to create new grid variant and add loads to terminals

In [ ]:
def create_load(pfi: PowerFactoryInterface, /, *, grid: PFTypes.Grid, load_data: dict[str, t.Any]) -> PFTypes.Load | None:
    logger = logging.getLogger()

    terminal = load_data.pop("terminal")
    load_name = load_data.pop("name")
    logger.info(f"Create new load '{load_name}' at terminal '{terminal.loc_name}' ...")

    term_cubic = terminal.GetContents("*." + PFClassId.CUBICLE.value)[0]
    # it is necessary to add the copy exactly on a bus, otherwise the field won't have the attribute 'cterm'
    new_cubicle = terminal.AddCopy(term_cubic, "NewField")
    data = {"bus1": new_cubicle}
    load_data.update(data)

    load = pfi.create_object(
        name=load_name,
        class_name=PFClassId.LOAD.value,
        location=grid,
        data=load_data,
    )
    return t.cast("PFTypes.Load", load) if load is not None else None

def routine_create_new_study_case_and_add_loads(
    pfi: PowerFactoryInterface,
    *,
    study_case_name: str,
    grid_variant_name: str,
    grid: PFTypes.Grid,
    loads_data: Sequence[dict[str, ValidPFValue]],
) -> Sequence[PFTypes.Load] | None:
    """Use this routine to create a new study case (based on the previously active one) and add new loads to given terminals.

    Args:
        study_case_name (str): new study case name
        grid_variant_name (str): new grid variant name
        grid (PFTypes.Grid): grid to which the loads belongs
        loads_data (Sequence[dict[str, ValidPFValue]]): list of dictionaries with load data

    Returns:
        Sequence[PFTypes.Load]: list of newly added loads
    """
    ## Create new study case and grid variant
    # Get active study case and grid variants
    active_sc = pfi.app.GetActiveStudyCase()
    default_active_variants = pfi.grid_variants(only_active=True)

    # Create new study case
    new_sc = pfi.study_case_dir.AddCopy(active_sc)  # type: ignore[reportArgumentType]
    new_sc.loc_name = study_case_name # type: ignore[reportOptionalMemberAccess]
    pfi.switch_study_case(study_case_name)

    # Create and activate new grid variant
    activation_time = int(math.floor(time.time()))  # now in seconds
    new_variant = pfi.create_grid_variant(name=grid_variant_name, data={"tAcTime": activation_time}, force=True)
    pfi.switch_grid_variant(grid_variant_name)

    ## Add new loads
    loads = [create_load(pfi, grid=grid, load_data=load_data) for load_data in loads_data]

    ## Reactivate old (already existing) and new grid variants
    pfi.deactivate_grid_variants()
    # reactivate old active variants
    for variant in default_active_variants:
        pfi.activate_grid_variant(variant)
    # reactivate new variant at the end
    pfi.activate_grid_variant(new_variant) # type: ignore[reportArgumentType]

    return pfi.filter_none(loads)

### Define routine to run Automatic Graph Layout Tool

In [ ]:
def run_layout(pfi: PowerFactoryInterface, /, *, selection: PFTypes.Selection | None = None) -> bool:
    """This function gets command GRAPHIC_LAYOUT_TOOL, adjusts and executes it."""
    logger = logging.getLogger()
    # Unfreeze grid diagram
    desktop = pfi.app.GetFromStudyCase(PFClassId.DESKTOP.value)
    if desktop is not None:
        desktop = t.cast("PFTypes.Desktop", desktop)
        desktop.Show()
        desktop.Unfreeze()
    else:
        logger.warning("Desktop set could not be loaded. Thus, graphic keeps frozen and automatic layouting does not work. Quitting ...")
        return False

    logger.debug("Start automatic layout applicaton ...")
    layout_cmd = pfi.app.GetFromStudyCase(CalculationCommand.GRAPHIC_LAYOUT_TOOL.value)
    layout_cmd = t.cast("PFTypes.CommandSglLayout", layout_cmd)
    layout_cmd.iAction = 1
    layout_cmd.orthoType = 1
    layout_cmd.nodeDispersion = 0
    layout_cmd.insertionMode = 0
    layout_cmd.neighborhoodSize = 3
    if selection is not None:
        layout_cmd.neighborStartElems = selection

    error = layout_cmd.Execute()
    if error:
        logger.warning("Automatic layouting failed.")
        return False
    return True

## Execute control action using a controller instance

In [ ]:
_project_name = PROJECT_NAME.split("\\")
full_export_path = pathlib.Path().cwd() / EXPORT_PATH / _project_name[-1]

# Configure logging to output to the notebook's standard output
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger()

with PowerFactoryInterface(
    powerfactory_service_pack=PF_SERVICE_PACK,
    powerfactory_user_profile=PF_USER_PROFILE,
    powerfactory_ini_name=PF_INI_NAME,
    python_version=PF_PYTHON_VERSION,
    project_name=PROJECT_NAME,
    logging_level=logging.INFO,
    # log_file_path=full_export_path / pathlib.Path("pf_control.log"),  # noqa: ERA001
) as pfi:
    logger.info("3_Bus : Run control example 'Add loads' ... ")

    study_case_name = "3_Bus"
    grid_name = "HV_3_Bus"

    # From active study case: get all relevant! terminals
    logger.info("3_Bus : From active study case: get all terminals ...")
    study_case = pfi.switch_study_case(study_case_name)
    grid = pfi.grid(grid_name)
    origin_data = pfi.compile_powerfactory_data(grid)  # type: ignore [reportArgumentType]
    terminals = origin_data.terminals

    # Specify load data (connected terminal, load name, further load data)
    loads_data = [{"name": "NewLoad1", "terminal": terminals[1], "plini": 1, "qlini": 0.5},
                  {"name": "NewLoad2", "terminal": terminals[2], "plini": 2, "qlini": 1}]

    loads = routine_create_new_study_case_and_add_loads(pfi, study_case_name="3_Bus NewLoadsAdded", grid_variant_name="NewLoadsAdded", grid=grid, loads_data=loads_data)

    logger.info("3_Bus : Run Automatic Layout Tool with newly created loads as selection.")
    # Create selection with all origin grid elements as base for the layout tool
    selection = pfi.create_sgl_layout_selection(data=terminals)

    # Run automatic layout tool
    success = run_layout(pfi, selection=selection)  # type: ignore [reportArgumentType]

    logger.info("3_Bus : Run control example 'Add loads' ... Done")